In [21]:
import os

import pandas as pd

from prepare_data.statistics import calculate_stats
from prepare_data.utils import create_folder

from sklearn.model_selection import train_test_split
import shutil

In [53]:
data_dir = '../data/clean_images94'
labels_dir = '../data/prepare_data/labels'
save_dir = '../data/yolov_data'

## Calculate Statistics for sep train - test - val

In [54]:
df = pd.DataFrame(columns=['file_name', 'path_to_image', 'path_to_label'])
for root, dirs, files in os.walk(data_dir):
    for file in files:
        path_to_image = os.path.join(root, file.replace('_label.', '.'))
        path_to_label = os.path.join(labels_dir, file.replace('_label.', '.').replace('jpg', 'txt'))
        df = pd.concat((df, pd.DataFrame({'file_name': file.split('.')[0],
                                          'path_to_image': path_to_image,
                                          'path_to_label': path_to_label}, index=[0])), ignore_index=True)

In [55]:
df

,file_name,path_to_image,path_to_label
0,8 (2)_label,../data/clean_images94/8 (2).jpg,../data/prepare_data/labels/8 (2).txt
1,7 (103)_label,../data/clean_images94/7 (103).jpg,../data/prepare_data/labels/7 (103).txt
2,12 (60)_label,../data/clean_images94/12 (60).jpg,../data/prepare_data/labels/12 (60).txt
3,4 (1)_label,../data/clean_images94/4 (1).jpg,../data/prepare_data/labels/4 (1).txt
4,7 (148)_label,../data/clean_images94/7 (148).jpg,../data/prepare_data/labels/7 (148).txt
...,...,...,...
172,5 (105)_label,../data/clean_images94/5 (105).jpg,../data/prepare_data/labels/5 (105).txt
173,12 (42)_label,../data/clean_images94/12 (42).jpg,../data/prepare_data/labels/12 (42).txt
174,1 (8)_label,../data/clean_images94/1 (8).jpg,../data/prepare_data/labels/1 (8).txt
175,5 (183)_label,../data/clean_images94/5 (183).jpg,../data/prepare_data/labels/5 (183).txt


In [56]:
stat = calculate_stats(df)

In [57]:
stat.groupby('video_id').sum()[['pat0', 'pat1', 'pat2', 'pat3', 'pat4']]

,pat0,pat1,pat2,pat3,pat4
video_id,,,,,
1,39,2,5,0,0
3,2,0,2,2,3
4,30,12,20,14,4
5,94,27,47,24,15
6,0,0,0,0,0
7,83,16,43,3,22
8,0,0,9,4,0
11,0,0,3,0,0
12,0,55,0,0,14


Труба - 12, 11, 9, 8, 6, 

In [58]:
tube = [12, 11, 9, 8, 6]
plane = [1, 3, 4, 5, 7]

In [59]:
train_tube, test_tube = train_test_split(tube, test_size=0.2, random_state=43, shuffle=True)
train_plane, test_plane = train_test_split(plane, test_size=0.2, random_state=43, shuffle=True)

train = train_plane + train_tube
test = test_plane + test_tube


In [60]:
test

[5, 8]

In [61]:
stat[stat['video_id'].isin(test)].groupby('video_id').sum()[['pat0', 'pat1', 'pat2', 'pat3', 'pat4']].sum()


pat0    94
pat1    27
pat2    56
pat3    28
pat4    15
dtype: object

In [62]:
stat[stat['video_id'].isin(train)].groupby('video_id').sum()[['pat0', 'pat1', 'pat2', 'pat3', 'pat4']].sum()
    

pat0    154
pat1     85
pat2     73
pat3     19
pat4     43
dtype: object

In [63]:
stat['train_test'] = stat['video_id'].map(lambda x: 'train' if x in train else 'test')

In [64]:
stat

,path_to_label,path_to_image,name,video_id,frame_id,pat0,pat1,pat2,pat3,pat4,train_test
0,../data/prepare_data/labels/8 (2).txt,../data/clean_images94/8 (2).jpg,8 (2)_label,8,2,0,0,2,1,0,test
1,../data/prepare_data/labels/7 (103).txt,../data/clean_images94/7 (103).jpg,7 (103)_label,7,103,4,1,1,0,1,train
2,../data/prepare_data/labels/12 (60).txt,../data/clean_images94/12 (60).jpg,12 (60)_label,12,60,0,1,0,0,0,train
3,../data/prepare_data/labels/4 (1).txt,../data/clean_images94/4 (1).jpg,4 (1)_label,4,1,0,0,1,0,1,train
4,../data/prepare_data/labels/7 (148).txt,../data/clean_images94/7 (148).jpg,7 (148)_label,7,148,6,0,2,0,0,train
...,...,...,...,...,...,...,...,...,...,...,...
172,../data/prepare_data/labels/5 (105).txt,../data/clean_images94/5 (105).jpg,5 (105)_label,5,105,0,1,0,0,0,test
173,../data/prepare_data/labels/12 (42).txt,../data/clean_images94/12 (42).jpg,12 (42)_label,12,42,0,0,0,0,0,train
174,../data/prepare_data/labels/1 (8).txt,../data/clean_images94/1 (8).jpg,1 (8)_label,1,8,7,0,0,0,0,train
175,../data/prepare_data/labels/5 (183).txt,../data/clean_images94/5 (183).jpg,5 (183)_label,5,183,2,1,0,0,0,test


In [65]:
path_to_save_yolov = r'../data/yolov_data/data94'

create_folder(os.path.join(path_to_save_yolov))
create_folder(os.path.join(path_to_save_yolov, "labels"))
create_folder(os.path.join(path_to_save_yolov, "labels", "train"))
create_folder(os.path.join(path_to_save_yolov, "labels", "test"))
create_folder(os.path.join(path_to_save_yolov, "labels", "val"))

create_folder(os.path.join(path_to_save_yolov, "images"))
create_folder(os.path.join(path_to_save_yolov, "images", "train"))
create_folder(os.path.join(path_to_save_yolov, "images", "test"))
create_folder(os.path.join(path_to_save_yolov, "images", "val"))


In [66]:
def create_yolov_dataset(data: pd.DataFrame, path_to_save: str):
    for i, row in data.iterrows():
        train_test = row['train_test']
                
        name = row['name']
        
        path_to_label = row['path_to_label']
        path_to_image = path_to_label.replace('labels', 'images').replace('.txt', '.jpg')


        shutil.copy(path_to_image, os.path.join(path_to_save, 'images', train_test, name + '.jpg'))
        shutil.copy(path_to_label, os.path.join(path_to_save, 'labels', train_test, name + '.txt'))
        
        if train_test == 'test':
            shutil.copy(path_to_image, os.path.join(path_to_save, 'images', 'val', name + '.jpg'))
            shutil.copy(path_to_label, os.path.join(path_to_save, 'labels', 'val', name + '.txt'))



In [67]:
create_yolov_dataset(stat, path_to_save_yolov)